<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/create_db_with_stem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def fetch_page_and_sublinks(url, max_sublinks=50):
    results = {}
    visited_urls = set()

    def fetch(url):
        if len(visited_urls) >= max_sublinks:
            return
        if url[-1] != "/":
          url += "/"
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            results[url] = soup
            visited_urls.add(url)
            # Find all links in the soup object
            for link in soup.find_all('a', href=True):
              suffix = link.get('href')
              if suffix == "#main":
                suffix = ""
              full_url = urljoin(url, suffix)
              if "azure" not in full_url:
                continue
              if full_url not in visited_urls:
                  fetch(full_url)  # Recursively fetch sublinks
                  if len(visited_urls) >= max_sublinks:
                    return
        else:
            results[url] = None

    fetch(url)
    return results

In [ ]:
# Example usage
url = "https://azure.microsoft.com/en-us"  # Replace with the URL you want to fetch
data = fetch_page_and_sublinks(url)
print(data.keys())  # This will print the URLs of the main page and sublinks fetched

dict_keys(['https://azure.microsoft.com/en-us/', 'https://portal.azure.com/', 'https://azure.microsoft.com/en-us/free/'])


In [3]:
stop_words = {'a', 'an', 'the','I', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
 'in', 'to', 'for', 'with', 'on', 'at', 'by', 'from', 'up', 'off', 'about', 'into', 'over', 'after',
 'and', 'but', 'or', 'as', 'if', 'when', 'than', 'because', 'while', 'where','be', 'have', 'do', 'is', 'am', 'are', 'was', 'were', 'being', 'been',
 'some', 'such', 'only', 'own', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'}

In [5]:
import re
def index_words(index, html_link, soup,stemmer):
  words = re.findall(r'\w+', soup.get_text())
  for word in words:
    word = word.lower()
    word = stemmer.stem(word)
    if word in stop_words:
      continue
    if word in index:
      index[word]["count"] += 1
      if html_link in index[word]["links"]:
        index[word]["links"][html_link] += 1
      else:
        index[word]["links"][html_link] = 1
    else:
      index[word] = {"count": 1, "links": {html_link:1}}
  return index

In [6]:
from nltk.stem import PorterStemmer
def create_index(url):
  stemmer = PorterStemmer()
  dic = fetch_page_and_sublinks(url)
  index = {}
  for html_link in dic:
    html_data = dic[html_link]
    index = index_words(index, html_link ,html_data,stemmer)
  #index = apply_stemming(index)
  return index


In [8]:
url = 'https://azure.microsoft.com/en-us'
index = create_index(url)
#print(index)
#print(index['bird'])
#print(index['john'])

In [9]:
import json
# The file path where you want to save the JSON data
file_path = 'my_data.json'

# Writing the dictionary to a file as JSON
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(index, f, ensure_ascii=False, indent=4)


In [ ]:
print(index['elected'])

{'count': 1, 'links': {'https://en.wikipedia.org/wiki/Main_Page': 1}}
